# Working comparing reciprocal and standard records and reading data

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 

from sys import argv

from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
from pyaspect.specfemio.write import *
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *

import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

## seting up dirs

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
proj_dirs = f'{data_out_dir}tmp/TestProjects/Computed_Forward_and_Reciprocity_Test'
standard_proj_dir = f'{proj_dirs}/ForwardTestProject'
reciprocal_proj_dir = f'{proj_dirs}/ReciprocalTestProject'

print('Standard Project Contents:')
print('--------------------------')
!ls {standard_proj_dir}
print()
print('Reciprocal Project Contents:')
print('--------------------------')
!ls {reciprocal_proj_dir}


## read records

In [ ]:
standard_records_fqp = os.path.join(standard_proj_dir,'pyheader.proj_records')
reciprocal_records_fqp = os.path.join(reciprocal_proj_dir,'pyheader.proj_records')

standard_records_h = read_records(standard_records_fqp)
reciprocal_records_h = read_records(reciprocal_records_fqp)

'''
''';
print(f'Standard Project Records:\n')
for record in standard_records_h:
    print(f'{len(record.get_solutions_header_list())}\n')
    print(f'{len(record.get_stations_header_list())}\n')

print(f'Reciprocal Project Records:\n')
for record in reciprocal_records_h:
    #print(f'{record}\n')
    print(f'{len(record.get_solutions_header_list())}\n')
    print(f'{len(record.get_stations_header_list())}\n')
    
recip_rec = reciprocal_records_h[0]

In [ ]:
idx = pd.IndexSlice
#print(recip_rec.get_solutions_header_list(key='depth',value=3.14))
r_df = recip_rec.solutions_df
#r_df = recip_rec.stations_df
#print(r_df.columns)
key = 'sid'
value = 1
#c_df = r_df.loc[r_df[key] == value]
#cidx = idx[slice(0),slice(0,1),slice(0,1),slice(0,7,2)]
#cidx = idx[slice(0),slice(0,1)]
tupix = (slice(0),slice(0,1))
#print('type(tupix):',type(tupix))
cidx = idx[tupix]
#cidx = idx[slice(0)]
#print(f'weird:\n{cidx}')

c_df = r_df.loc[cidx,:]
#print(f'r_df:\n{r_df}\n')
#print(f'c_df:\n{c_df}')

print()
print('-------------------- SLICED -------------------------------------------')
#s_rec = recip_rec[:,:,::2,::2]
s_rec = recip_rec.copy()
#print(s_rec)
print('get_val:',s_rec.solutions_df.index.get_level_values('sid').nunique)
print('level:',s_rec.solutions_df.index.levels[0])
print()
s_rec.solutions_df.reset_index()
print('count:',len(s_rec.stations_df))
print(dir(s_rec.stations_df.index))
print('series:',s_rec.stations_df.index.to_series().nunique())


In [ ]:
src_df = recip_rec.solutions_df.copy()
rec_df = recip_rec.stations_df.copy()

print(f'Source DF:\n{src_df}')
print('\n\n')
print(f'Receiver DF:\n{rec_df}')

print('\n\n')
print('------------------------------- Renamed ---------------------------------------')

#src_df.rename(columns = {'lat_yc': 'src.lat_yc', 'lon_xc': 'src.lon_xc'}, inplace=True)
#src_df.columns = ['src.' + col if col != 'col1' and col != 'col2' else col for col in df.columns]
src_df.reset_index(inplace=True)
src_df.set_index(['proj_id','eid','sid'],inplace=True)
src_df.columns = 'src.' + src_df.columns
rec_df.reset_index(inplace=True)
rec_df.set_index(['proj_id','eid','sid','trid','gid'],inplace=True)
rec_df.columns = 'rec.' + rec_df.columns

#new_df = rec_df + src_df
idx = pd.IndexSlice

print(f'Source DF:\n{src_df}')
print('\n\n')
print(f'Receiver DF:\n{rec_df}')
print('\n\n')
print(f'slice rec DF:\n{rec_df.loc[idx[0,0,0,:,:],:]}')

In [ ]:
class TRac():
    def __init__(self):
        
        myzeros  = np.arange(10)
        myones   = 10 + myzeros.copy()
        mytwos   = 10 + myones.copy()
        mythrees = 10 + mytwos.copy()
        
        self.data = {'zero':myzeros,'one':myones,'two':mytwos,'three':mythrees}
        
    def append(self, item):
        self.data.append(item)
        
    def __getitem__(self, *kslice):
        print('len(kslice):',len(kslice))
        print('kslice:',kslice)
        print('type(kslice[0]):',type(kslice[0]))
        rval = None
        if isinstance(*kslice,slice):
            print('is instance')
            j = 0
            if 1 < len(*kslice) and len(*kslice) <= 2:
                j = 1
            else:
                raise Exception(f'too many slice indices')
                
            rval = []
            for i in range(kslice[0].start,kslice[0].stop,kslice[0].step):
                print('i:',i)
                rval.append(self.data[list(self.data.keys())[i]][kslice[j].start:kslice[j].stop:kslice[j].step])
                
        elif isinstance(kslice[0],str):
            rval = self.data[kslice]
        else:
            raise Exception(f'incorrect index type')
            
        return rval 
    
    
trac = TRac()
#x = trac[0:6:2,2:4,0:3,1:3]
#x = trac['depth']
#x = trac[0:4:2,2:4]
#x = trac[1:3:2,0:9:3]
x = trac[1:3:2]
print(x)
print()
y = trac['three']
print(y)
print()

#print(x[0])
#print()
#print(dir(x[0]))
#print('start:',x[0].start)
#print('stop: ',x[0].stop)
#print('step: ',x[0].step)

In [ ]:
def test_func(*args):
    print(len(args))
    
test_func(slice(0))

In [ ]:
#yt = (1,2,3,4,5,6)
yt = (1,)
print(yt[0:2])